In [1]:
import sys
sys.path.append('..')
from somegrad import Tensor
import somegrad.functional as F

In [2]:
def build_dataset(words, block_size):

    xs, ys = [], []

    for w in words:
        context = [0] * block_size
        
        for ch in w + '.':
            ix = stoi[ch]
            xs.append(context)
            ys.append(ix)

            context = context[1:] + [ix] # crop and append
    
    xs = Tensor(xs) 
    ys = Tensor(ys)
    
    return xs, ys 

In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [4]:
np.random.seed(42)

n_embd = 30
n_hidden = 200
block_size = 3

C = Tensor(np.random.randn(27, n_embd))

W1 = Tensor(np.random.randn(block_size * n_embd, n_hidden) * 0.2)
b1 = Tensor(np.random.randn(n_hidden) * 0.01) # Biases should be close to zero
W2 = Tensor(np.random.randn(n_hidden, 27) * 0.01) # The output layer should be small
b2 = Tensor(np.zeros(27)) # Initializing biases to 0
parameters = [C, W1, b1, W2, b2]

In [ ]:
words = open('names.txt', 'r').read().splitlines()

# build indexing for characters
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

random.seed(42)
random.shuffle(words)

n = len(words)
n1 = int(0.8 * n)
n2 = int(0.9 * n)

train_words = words[:n1]      # 80%
dev_words   = words[n1:n2]    # 10%
test_words  = words[n2:]      # 10%

print(f"Split sizes: Train: {len(train_words)}, Dev: {len(dev_words)}, Test: {len(test_words)}")

# create datasets
Xtr, Ytr   = build_dataset(train_words, block_size)
Xdev, Ydev = build_dataset(dev_words, block_size)
Xte, Yte   = build_dataset(test_words, block_size)

Split sizes: Train: 25626, Dev: 3203, Test: 3204


In [6]:
lre = np.linspace(-1, -2.5, 200000)
lrs = 10**lre
lri = []
lossi = []

In [7]:
for i in range(200000):

    # minibatch construct
    ix = np.random.randint(0, Xtr.data.shape[0], (32,))

    # forward pass
    emb = C[Xtr[ix].data]
    h = (emb.reshape(-1, block_size * n_embd) @ W1 + b1).tanh()
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = np.zeros_like(p.data)
    loss.backward()

    # update
    lr = lrs[i]
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    # lri.append(lre[i])
    # lossi.append(loss.data.item())

In [8]:
emb = C[Xtr.data]
h = (emb.reshape(-1, block_size * n_embd) @ W1 + b1).tanh()
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr.data)
loss

Tensor(data=1.99830986872439, grad=0.0)

In [9]:
emb = C[Xdev.data]
h = (emb.reshape(-1, block_size * n_embd) @ W1 + b1).tanh()
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev.data)
loss

Tensor(data=2.083173390205829, grad=0.0)

In [10]:
emb = C[Xte.data]
h = (emb.reshape(-1, block_size * n_embd) @ W1 + b1).tanh()
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yte.data)
loss

Tensor(data=2.08300039701566, grad=0.0)

In [11]:
# sample from the model

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      input_idx = np.array([context]) 
      emb = C[input_idx] # (1, block_size, n_embd)
      h = (emb.reshape(1, -1) @ W1 + b1).tanh()
      logits = h @ W2 + b2

      # softmax
      counts = logits.exp()
      probs = counts / counts.sum(axis=1, keepdims=True)

      # sampling
      p = probs.data[0]
      p = p / p.sum() 
      
      ix = np.random.choice(len(p), p=p)
      
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

sheer.
kanne.
aleslynn.
saine.
pain.
demeryna.
dayrahni.
keiro.
skyla.
kariyanahyaireliailin.
ajsadianee.
adalas.
ashde.
yoir.
yva.
hiyah.
azalynn.
emorgasei.
clochel.
lak.
